# HW1 SOLUTION : [Landon McAllister | 913335772]

### (80%) Perform basic data engineering on the light pollution dataset.


For our data engineering we want to broadly
complete the following:

1. combine all the GaN files into a single file and thus a
   single Pandas DataFrame,
2. convert specific data fields to numeric fields using
   Pandas,
3. combine date and time fields for easier time series evaluation,
3. split data out of specific fields and into new simpler
   fields,
4. use Pandas to understand the underlying distrubution
   of certain data.




**&#167; Task:**  **Use the file of URLs from the first assignment to load the data from 2014-2024 and store it all into a single file.**


In [34]:
def assign_location(d):
    """
    Reduces a string to one of four categories, depending on whether "urban", "suburban", "rural" or "remote" is in the string.

    Args:
        d (str): A string or sentence containing one or more of the target words.

    Returns:
        str: the string target or None if not foun
    """
    import re

    d = str(d).lower()
    re_urban = r"\w*(?<![Ss]ub)urban"

    if re.match(re_urban, d):
        return "urban"
    elif "suburban" in d:
        return "suburban"
    elif "rural" in d:
        return "rural"
    elif "remote" in d:
        return "remote"

In [35]:
import pandas as pd

with open("/gan_urls.txt","r", encoding="utf-8") as f:
    urls = [line.strip() for line in f if line.strip()]

    dfs=[pd.read_csv(url) for url in urls]
    df=pd.concat(dfs,ignore_index=True)

    df = df.drop_duplicates().reset_index(drop=True)

    df.to_csv("2014_to_2024_gan_data.csv", index=False)
print("Combined dataset saved as 2014_to_2024_gan_data.csv")

Combined dataset saved as 2014_to_2024_gan_data.csv


**&#167; Task:**  **Convert categorical fields to numeric ones.**

We will convert these to numeric values using the following guidance:

* "clear" &#8594; 0
* "1/4 of sky" &#8594; 0.25
* "1/2 of sky" &#8594; 0.50
* "over 1/2 of sky" &#8594; 0.75

In [36]:
cloud_map={
    "clear": 0,
    "1/4 of sky": 0.25,
    "1/2 of sky": 0.50,
    "over 1/2 of sky": 0.75
}

df["CloudCoverPct"] = df["CloudCover"].map(cloud_map)

**&#167; Task:**  **Convert `Constellation` to numeric binary values (binarize the column).**


In [37]:
const_dummies = pd.get_dummies(df["Constellation"], prefix="const")
df = pd.concat([df, const_dummies], axis=1)

**&#167; Task:**  **Create 5 binary columns `loc_urban`, `loc_suburban`, `loc_rural`, `loc_remote` and `loc_unknown`.**

In [38]:
df["LocationComment"] = df["LocationComment"].fillna("unknown")
df["LocationCategory"] = df["LocationComment"].apply(assign_location)

loc_dummies = pd.get_dummies(df["LocationCategory"], prefix="loc")

df = pd.concat([df, loc_dummies], axis=1)

**&#167; Task:**  **Merge `LocalDate` and `LocalTime` and convert to a `datetime` object.**


In [39]:
df["LocalDateTime"] = pd.to_datetime(df["LocalDate"] + " " + df["LocalTime"], errors="coerce")

**&#167; Task:**  **Convert missing `SQMReading` values to `-1`.**

In [40]:
df["SQMReading"] = df["SQMReading"].fillna(-1)

**&#167; Task:**  **Store a new dataset file with a reduced set columns which include our numeric ones.**

In [41]:
keep_cols = [
    "Latitude", "Longitude", "Elevation(m)", "LocalDateTime",
    "LimitingMag", "SQMReading", "CloudCoverPct",
    "const_Bootes", "const_Canis Major", "const_Crux", "const_Cygnus",
    "const_Gemini", "const_Grus", "const_Hercules", "const_Leo",
    "const_None", "const_Orion", "const_Pegasus", "const_Perseus",
    "const_Sagittarius", "const_Scorpius", "const_Taurus",
    "loc_remote", "loc_rural", "loc_suburban", "loc_unknown", "loc_urban"
]

for col in keep_cols:
  if col not in df.columns:
    df[col]=0

df[keep_cols].to_csv("2014_to_2024_gan_data_working.csv", index=False)
print("Final working dataset saved as 2014_to_2024_gan_data_working.csv")

Final working dataset saved as 2014_to_2024_gan_data_working.csv
